In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

In [ ]:
train_dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

In [ ]:
train_loader =  DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False
)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # 🔹 Primeiro bloco convolucional
        self.conv1 = nn.Conv2d(
            in_channels=3,    # RGB
            out_channels=32,
            kernel_size=3,
            padding=1
        )
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        # 🔹 Segundo bloco convolucional
        self.conv2 = nn.Conv2d(
            in_channels=32,
            out_channels=64,
            kernel_size=3,
            padding=1
        )
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        # 🔹 Flatten
        self.flatten = nn.Flatten()

        # 🔹 Camadas totalmente conectadas
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Entrada: (batch, 3, 32, 32)

        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        # → (batch, 32, 16, 16)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        # → (batch, 64, 8, 8)

        x = self.flatten(x)
        # → (batch, 4096)

        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)

        return x

In [ ]:
model = SimpleCNN().to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(30):
    print("training epoch: ", epoch)
    running_loss = 0.0
    model.train()
    

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print("loss:", running_loss/len(train_loader))

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model.eval()

dataiter = iter(test_loader) 
images, labels = next(dataiter)

img = images[1]
label = labels[1]

img_input = img.unsqueeze(0).to(device)

with torch.no_grad():
    output = model(img_input)
    _, predicted = torch.max(output, 1)

print(f'Rótulo Real: {classes[label]}')
print(f'Predição do Modelo: {classes[predicted.item()]}')